#### 추천시스템(Recommend System)
* 추천 시스템은 크게 두 가지로 구분 가능
    - 컨텐츠 기반 필터링
    - 협업 필터링
* 두 가지를 조합한 hybrid 방식 가능
* 컨텐츠 기반 필터링은 지금까지 사용자의 이전 행동과 명시적 피드백을 통해 사용자가 좋아하는 것과 유사한 항목을 추천
* 협업 필터링은 사용자와 항목간의 유사성을 동시에 사용해 

#### Surprise
* 추천 시스템 개발을 위한 라이브러리
* 다양한 모델과 데이터 제공
* scikit-learn과 유사한 사용 방법

In [ ]:
# Microsoft Visual C++ 14.0 or greater is required

from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate


In [ ]:
data = Dataset.load_builtin('ml-100k', prompt = False)
data.raw_ratings[:10] # ['아이디(사용자)', '아이템(영화)', '평점', '????' ]

In [ ]:
model = SVD()
cross_validate(model, data, measures=['rmse','mae'], cv = 5, verbose= True)

#### 컨텐츠 기반 필터링(Content-based Filtering)
* 컨텐츠 기반 필터링은 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천
    * ex) 내가 지금까지 시청한 영화 목록과 다른 사용자의 시청 목록을 비교해 나와 비슷한 취향의 사용자가 시청한 영화를 추천
* 유사도를 기반으로 추천
* 컨텐츠 기반 필터링은 다음과 같은 장단점이 있음
    * 장점
        * 많은 수의 사용자를 대상으로 쉽게 확장 가능
        * 사용자가 관심을 갖지 않던 상품 추천 가능
    * 단점
        * 입력 특성을 직접 설계해야 하기 때문에 많은 도메인 지식이 필요
        * 사용자의 기존 관심사항을 기반으로만 추천 가능

In [ ]:
import numpy as np
from surprise import Dataset

In [ ]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)

raw_data[:,0] -= 1
raw_data[:,1] -= 1

n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:, 1])

shape = (n_users +1, n_movies +1)
shape

In [ ]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id] = 1.
adj_matrix

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        similarity = np.dot(my_vector, user_vector)
        if similarity > best_match:
            best_match = similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match : {}, Best Match ID : {}'.format(best_match, best_match_id))

In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:
        recommend_list.append(i)

print(recommend_list)

#### 유클리드 거리를 사용해 추천
* 거리가 가까울 수록(값이 작을 수록) 나와 유사한 사용자

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
        if euclidean_dist < best_match:
            best_match = euclidean_dist
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match : {}, Best Match ID : {}'.format(best_match, best_match_id))

In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:
        recommend_list.append(i)

print(recommend_list)

#### 코사인 유사도를 사용해 추천
* 두 벡터가 이루고 있는 각을 계산

In [17]:
# 코사인 유사도 계산 함수 만들기
def compute_cos_similarity(v1, v2):
    norm1 = np.sqrt(np.sum(np.square(v1)))
    norm2 = np.sqrt(np.sum(np.square(v2)))
    dot = np.dot(v1, v2)
    return dot / (norm1 * norm2)

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match : {}, Best Match ID : {}'.format(best_match, best_match_id))

In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:
        recommend_list.append(i)

print(recommend_list)

#### 기존 방법(안봤는지, 봤는지(0,1))에 명시적 피드백(사용자가 평가한 영화 점수)을 추가해 실험

In [ ]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id] = rating
adj_matrix

In [ ]:
# 유클리디언으로
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
        if euclidean_dist < best_match:
            best_match = euclidean_dist
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match : {}, Best Match ID : {}'.format(best_match, best_match_id))

In [ ]:
# 코사인유사도로
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match : {}, Best Match ID : {}'.format(best_match, best_match_id))

#### 협업 필터링(Collaborative Filtering)
* 사용자와 항목의 유사성을 동시에 고려해 추천
* 기존에 내 관심사가 아닌 항목이라도 추천 가능
* 자동으로 임베딩 학습 가능
* 협업 필터링은 다음과 같은 장단점을 갖고 있음
    * 장점
        * 자동으로 임베딩을 학습하기 때문에 도메인 지식이 필요 없음
        * 기존의 관심사가 아니더라도 추천 가능
    * 단점
        * 학습 과정에 나오지 않은 항목은 임베딩을 만들 수 없음
        * 추가 특성을 사용하기 어려움

In [ ]:
from surprise import KNNBasic, SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate

In [ ]:
data = Dataset.load_builtin('ml-100k', prompt=False)

In [ ]:
# KNN을 사용한 협업 필터링
model = KNNBasic()
cross_validate(model, data, measures=['rmse','mae'], cv=5, n_jobs=4, verbose=True)

In [ ]:
# SVD를 사용한 협업 필터링
model = SVD()
cross_validate(model, data, measures=['rmse','mae'], cv=5, n_jobs=4, verbose=True)

In [ ]:
# SVD를 사용한 협업 필터링
model = NMF()
cross_validate(model, data, measures=['rmse','mae'], cv=5, n_jobs=4, verbose=True)

In [ ]:
# SVD++를 사용한 협업 필터링
model = SVDpp()
cross_validate(model, data, measures=['rmse','mae'], cv=5, n_jobs=4, verbose=True)

#### 하이브리드
* 컨텐츠 기반 필터링과 협업 필터링을 조합한 방식
* 많은 하이브리드 방식이 존재
* 실습에서는 협업 필터링으로 임베딩을 학습하고 컨텐츠 기반 필터링으로 유사도 기반 추천을 수행하는 추천 엔진 개발

In [8]:
import numpy as np
from sklearn.decomposition import randomized_svd, non_negative_factorization 
from surprise import Dataset

In [9]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)

raw_data[:,0] -= 1
raw_data[:,1] -= 1

n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:, 1])

shape = (n_users +1, n_movies +1)
shape

(943, 1682)

In [10]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id] = rating
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [11]:
U, S, V = randomized_svd(adj_matrix, n_components=2) # U사용자, S특이값벡터 ,V아이템
S = np.diag(S)

d:\dev\git\recommender-system\.venv\lib\site-packages\sklearn\utils\extmath.py:368: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  warnings.warn(


In [12]:
print(U.shape)
print(S.shape)
print(V.shape)

(943, 2)
(2, 2)
(2, 1682)


In [15]:
# 분해 된걸 다시 결합(복원)
np.matmul(np.matmul(U, S), V) 

array([[ 3.91732663e+00,  1.47276644e+00,  7.98261988e-01, ...,
         6.24907189e-04,  1.41100852e-02,  1.36545878e-02],
       [ 1.85777226e+00,  3.96191175e-01,  5.05705740e-01, ...,
         5.38862978e-03,  1.77237914e-03,  5.26968095e-04],
       [ 8.94989517e-01,  1.71578497e-01,  2.51738682e-01, ...,
         2.92094923e-03,  5.39937171e-04, -1.25733753e-04],
       ...,
       [ 9.92051955e-01,  2.10814957e-01,  2.70363365e-01, ...,
         2.89019297e-03,  9.34221962e-04,  2.66612193e-04],
       [ 1.30425401e+00,  5.27669941e-01,  2.50080165e-01, ...,
        -4.20677765e-04,  5.30525683e-03,  5.28069948e-03],
       [ 2.82999397e+00,  9.70812247e-01,  6.15871694e-01, ...,
         2.02091492e-03,  8.67740813e-03,  8.03107892e-03]])

* 사용자 기반 추천
* 나와 비슷한 취향을 가진 다른 사용자의 행동 추천
* 사용자 특징 벡터의 유사도 사용

In [18]:
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(U):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match : {}, Best Match ID : {}'.format(best_match, best_match_id))

Best Match : 0.9999942295956324, Best Match ID : 235


In [20]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:
        recommend_list.append(i)

print(recommend_list)

[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


* 항목 기반 추천
* 내가 본 항목과 비슷한 항목을 추천
* 항목 특징 벡터의 유사도 사용

In [30]:
my_id, my_vector = 0, V.T[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(V.T):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match : {}, Best Match ID : {}'.format(best_match, best_match_id))

Best Match : 0.9999999951364145, Best Match ID : 1287


In [22]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
    if adj_matrix[i][my_id] > 0.9 :
        recommend_list.append(i)

print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 

* 비음수 행렬 분해를 사용한 하이브리드 추천

In [32]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [33]:
A, B, iter = non_negative_factorization(adj_matrix, n_components= 2)

d:\dev\git\recommender-system\.venv\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


In [34]:
np.matmul(A, B)

array([[3.71108858e+00, 1.48454102e+00, 7.39535363e-01, ...,
        3.64490531e-03, 1.45506510e-02, 1.44110916e-02],
       [2.11724416e+00, 2.37338725e-01, 5.51650703e-01, ...,
        4.76092332e-03, 3.03585167e-05, 0.00000000e+00],
       [9.85272360e-01, 1.10447010e-01, 2.56713988e-01, ...,
        2.21552443e-03, 1.41275191e-05, 0.00000000e+00],
       ...,
       [1.04485667e+00, 1.17126289e-01, 2.72238757e-01, ...,
        2.34950819e-03, 1.49818804e-05, 0.00000000e+00],
       [1.45758857e+00, 5.42112787e-01, 2.99181695e-01, ...,
        1.61177659e-03, 5.15921394e-03, 5.10771005e-03],
       [2.44733103e+00, 9.41208708e-01, 4.95740038e-01, ...,
        2.56992125e-03, 9.08287324e-03, 8.99389282e-03]])

* 사용자 기반 추천

In [35]:
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(U):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match : {}, Best Match ID : {}'.format(best_match, best_match_id))

Best Match : 0.9999942295956324, Best Match ID : 235


In [36]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
    log1, log2 = log
    if log1 < 1. and log2 > 0.:
        recommend_list.append(i)

print(recommend_list)

[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


* 항목 기반 추천

In [37]:
my_id, my_vector = 0, V.T[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(V.T):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('Best Match : {}, Best Match ID : {}'.format(best_match, best_match_id))

Best Match : 0.9999999951364145, Best Match ID : 1287


In [38]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
    if adj_matrix[i][my_id] > 0.9 :
        recommend_list.append(i)

print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 